In [ ]:
from pathlib import Path
from moviepy import ImageClip, CompositeVideoClip
from moviepy.video import fx as vfx

base = Path("/Users/andy") / "Google Drive" / "My Drive" / "Projects" / "terra"

BG_FILE    = base / "figures" / "ak_dem_8x.png"
CLOUD_FILE = base / "figures" / "clouds.jpg"
LOGO_FILE  = base / "figures" / "pism_logo.png"

duration    = 6.0      # total video length (s)

cloud_start = 0.5       # when clouds start appearing
cloud_in    = 1.5       # cloud fade-in duration
cloud_hold  = 2.0       # how long clouds stay fully visible
cloud_out   = 1.5       # cloud fade-out duration

logo_start  = 2.0       # when logo starts to appear
logo_in     = 1.5       # logo fade-in duration

# ---- background ----
bg = ImageClip(str(BG_FILE)).with_duration(duration)

# Force even width/height for H.264 / yuv420p
w_even = bg.w if bg.w % 2 == 0 else bg.w - 1
h_even = bg.h if bg.h % 2 == 0 else bg.h - 1
bg = bg.resized(new_size=(w_even, h_even))

# ---- clouds layer ----
cloud = (
    ImageClip(str(CLOUD_FILE))
    .resized(width=bg.w * 1.3)      # a bit larger than frame
    .with_duration(cloud_in + cloud_hold + cloud_out)
    .with_start(cloud_start)        # appear over the DEM
    .with_opacity(0.9)
)

def cloud_pos(t):
    # t is local time since cloud_start
    x = bg.w / 2 + 60 * (t / cloud.duration)   # drift slightly right
    y = bg.h / 2 - 20 * (t / cloud.duration)   # and slightly up
    return (x - cloud.w / 2, y - cloud.h / 2)

cloud = cloud.with_position(cloud_pos)
cloud = cloud.with_effects([vfx.CrossFadeIn(cloud_in), vfx.CrossFadeOut(cloud_out)])

# ---- logo base clip (no position yet) ----
logo_base = (
    ImageClip(str(LOGO_FILE))
    .resized(height=int(bg.h * 0.15))          # scale relative to frame
    .with_duration(duration - logo_start)
    .with_start(logo_start)
)

# Where you want the logo on screen
logo_pos = (100, 100)

# ---- shadow for logo ----
shadow_offset = (8, 8)  # pixels (x, y) offset of the shadow

shadow = (
    logo_base
    .resized(height=int(150))          # scale relative to frame
    .with_position((logo_pos[0] + shadow_offset[0],
                    logo_pos[1] + shadow_offset[1]))
    .with_opacity(0.4)                    # semi-transparent
    .with_effects([vfx.CrossFadeIn(logo_in)])  # fade in with the logo
)
logo_height = int(bg.h * 0.075)   # was 0.15 → smaller

logo_base = (
    ImageClip(str(LOGO_FILE))
    .resized(height=logo_height)
    .with_duration(duration - logo_start)
    .with_start(logo_start)
)

logo_pos = (75, 75)
shadow_offset = (8, 8)

shadow = (
    logo_base
    .with_position((logo_pos[0] + shadow_offset[0],
                    logo_pos[1] + shadow_offset[1]))
    .with_opacity(0.4)
    .with_effects([vfx.CrossFadeIn(logo_in)])
)

logo = (
    logo_base
    .with_position(logo_pos)
    .with_effects([vfx.CrossFadeIn(logo_in)])
)


# ---- composite & export ----
final = CompositeVideoClip([bg, cloud, shadow, logo], size=bg.size)

final.write_videofile(
    base / "animation" / "ak_cloud_logo.mp4",
    fps=30,
    codec="libx264",
    audio=False,
    ffmpeg_params=["-pix_fmt", "yuv420p"],
)

# last frame as PNG (assuming fps is defined)
fps = 30
final.save_frame(
    base / "animation" / "ak_cloud_logo.png",
    t=final.duration - 1.0 / fps,
)
